<a href="https://colab.research.google.com/github/vedantdave77/project.Orca/blob/master/Project/project-Dog_Breed_CNN_Classification/Dog_Breed_App.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Previously, I implement my agorithm and it has very poor accuracy, so I will use transfer learning to get best result. and Deploy App flow. 

In [40]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import numpy as np
from glob import glob

# load filenames for human and dog images
human_files = np.array(glob("/content/drive/My Drive/Data /lfw/*/*"))
dog_files = np.array(glob("/content/drive/My Drive/Data /dogImages/*/*/*"))

human_files_short = human_files[:100]                            # pick first 100 images for testing
dog_files_short = dog_files[:100]                                # pick first 100 images for testing pretrained detector 

### Set Requried Parameters 

In [3]:
# set parameters which I usually used during project 
norm_mean = [0.485, 0.456, 0.406]
norm_std = [0.229, 0.224, 0.225]
img_short_side_resize = 256
img_input_size = 224
shuffle = True
num_workers = 16
batch_size = 64

### Data Loader [Access / Load/ Tranfer]

In [4]:
# import required libraries 
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from PIL import Image
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True                                          # cutting image (specially, short of edge)

import torch
import torchvision.transforms as transforms
from torchvision import datasets 

# define data_transformation and batch_size 
transform_train = transforms.Compose([
                                      transforms.Resize(img_short_side_resize),
                                      transforms.ColorJitter(brightness = 0.2, contrast = 0.2, saturation =0.2, hue = 0.1),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.RandomResizedCrop(img_input_size, scale=(0.08,1), ratio = (1,1)),
                                      transforms.ToTensor(),
                                      transforms.Normalize(mean = norm_mean, std = norm_std)
                                       ])

transform_test = transforms.Compose([
                                     transforms.Resize(img_input_size),
                                     transforms.FiveCrop(img_input_size),
                                     transforms.Lambda(lambda crops: torch.stack([
                                                                                  transforms.Compose([
                                                                                                      transforms.ToTensor(),
                                                                                                      transforms.Normalize(mean = norm_mean, std = norm_std)])(crop) for crop in crops
                                                                                  ]))

                                    ])

# load data (define datasets)
train_data = datasets.ImageFolder("/content/drive/My Drive/Data /dogImages/train/",transform_train)
valid_data = datasets.ImageFolder("/content/drive/My Drive/Data /dogImages/valid/",transform_test)
test_data  = datasets.ImageFolder("/content/drive/My Drive/Data /dogImages/test/",transform_test)

# separate imput and labels(classes)
data = {"train" : train_data, "valid" : valid_data, "test" : test_data}
n_classes = len(train_data.classes)

# create loaders (train, valid, test)
train_loader = torch.utils.data.DataLoader(data["train"], batch_size = batch_size, num_workers = num_workers, shuffle = shuffle, pin_memory = True)
valid_loader = torch.utils.data.DataLoader(data["valid"], batch_size = int(np.floor(batch_size/5)), num_workers=0, shuffle = shuffle, pin_memory = True) 
test_loader = torch.utils.data.DataLoader(data["test"], batch_size = int(np.floor(batch_size/5)), num_workers=0, shuffle = shuffle, pin_memory = True)

# loader dictionary
loaders_dict = {"train" : train_loader, "valid" : valid_loader, "test" : test_loader}

## Model Architecture (pretrained Transfer Learning Model) 


In [5]:
import torchvision.models as models 
import torch.nn as nn

# specify model 
model_tl = models.resnet50(pretrained = True)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/checkpoints/resnet50-19c8e357.pth


### Adjust parameters 

In [6]:
for param in model_tl.parameters():
    param.requires_grad = False                                                 # pretrained model does not need backpropagation. 

### CUDA checking 

In [7]:
use_cuda = torch.cuda.is_available()
if not use_cuda:
    print('CUDA is not available.  Training on CPU ...')
    device = "cpu"
else:
    print('CUDA is available!  Training on GPU ...')
    device = torch.device("cuda:0")
    print("Using",torch.cuda.get_device_name(device))


CUDA is not available.  Training on CPU ...


### Model Customization  [Model is already defined]

In [8]:
# modifying last layer as per custome requirements
model_tl.fc = nn.Linear(model_tl.fc.in_features,n_classes)

# initializer the weight for new layers
nn.init.kaiming_normal_(model_tl.fc.weight, nonlinearity = 'relu')

# transfer data and model to CUDA (Tesla P4)
model_tl = model_tl.to(device)

### Specify Loss and Optimizer 

In [9]:
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau

criterion_tl = nn.CrossEntropyLoss()
optimizer_tl = optim.Adam(model_tl.parameters(),3e-4)
scheduler_tl = ReduceLROnPlateau(optimizer_tl,'min',verbose=True, factor = 0.5, patience =7)

## Train and Validation of Model

---
### Model training and training loss

In [10]:
import time 
def train_epoch(model,train_loader,optimizer,criterion,device):
    train_loss = 0.0
    model.train()                                                               # define training job
    for batch_idx, (data,target) in enumerate(train_loader):                                                                     
        data, target = data.to(device),target.to(device)                        # move to CUDA
        optimizer.zero_grad()                                                   # reset gradient 
        output = model(data)                                                    # run model and get output 
        loss = criterion(output, target)                                        # calculate loss
        train_loss += loss.item() * data.size(0)                                # calculate gradients
        loss.backward()                                                         # update each layer para value
        optimizer.step()
    train_loss = train_loss/len(train_loader.dataset)
    return model, train_loss

### Validation epoch loss 

In [11]:
def valid_epoch(model,valid_loader,criterion,device,fivecrop): 
    valid_loss = 0.0
    model.eval()
    with torch.no_grad():
        for data, target in valid_loader:
            data, target = data.to(device), target.to(device)                   # tranfer to CUDA Tesla P 100
            if fivecrop == "mean":
                bs, ncrops, c, h, w = data.size()                               # dimenstion due to 5 crop method              
                output = model(data.view(-1,c,h,w))
                output = output.view(bs, ncrops, -1).mean(1)
            elif fivecrop == "max":
                bs, ncrops, c,h,w = data.size()
                output = model(data.view(-1,c,h,w))
                output = output.view(bs, ncrops, -1).max(1)[0] 
            else:
                output = model(data)
            
            loss = criterion(output, target)                                    # update losses
            valid_loss += loss.item() * data.size(0)
    valid_loss = valid_loss/len(valid_loader.dataset)
    return valid_loss    

In [12]:
def train(n_epochs, loaders_dict, model,optimizer, criterion, device, path_model, fivecrop = None, lr_scheduler = None):
    valid_loss_min = np.Inf
    train_loss = []
    valid_loss = []
    # time everything 
    time_start = time.time()
    for epoch in range(1, n_epochs+1): 
        time_start_epoch = time.time()
        
        model, train_loss_epoch = train_epoch(model,loaders_dict["train"],optimizer,criterion,device)
        train_loss.append(train_loss_epoch)

        # validate this epoch 
        valid_loss_epoch = valid_epoch(model,loaders_dict["valid"],criterion, device, fivecrop)

        # call learning rate scheduler 
        if lr_scheduler is not None:
            lr_scheduler.step(valid_loss_epoch)
        valid_loss.append(valid_loss_epoch)
        
        # save model for new lowest validation loss 
        if valid_loss_epoch <= valid_loss_min:
            torch.save(model.state_dict(),path_model)
            valid_loss_min = valid_loss_epoch

        print("Epoch {} done in {:.2f} seconds. \t | Training Loss: {:.3f} \t | Validation Loss: {:.3f}".format(
            epoch, time.time() - time_start_epoch, train_loss_epoch, valid_loss_epoch))
    print(f"{n_epochs} epochs ready in {(time.time() - time_start):.3f} seconds. Minimum validation loss: {valid_loss_min:.3f}")
    model.load_state_dict(torch.load(path_model))
    return model


### Model_Training Job Execution

In [28]:
model_tl = train(100, loaders_dict, model_tl, optimizer_tl, criterion_tl, device, 'model_xlearning.pt', fivecrop = 'mean', lr_scheduler = scheduler_tl)


Epoch 1 done in 998.36 seconds. 	 | Training Loss: 4.053 	 | Validation Loss: 2.705
Epoch 2 done in 147.71 seconds. 	 | Training Loss: 2.517 	 | Validation Loss: 1.590
Epoch 3 done in 148.60 seconds. 	 | Training Loss: 1.864 	 | Validation Loss: 1.129
Epoch 4 done in 149.03 seconds. 	 | Training Loss: 1.524 	 | Validation Loss: 0.900
Epoch 5 done in 148.49 seconds. 	 | Training Loss: 1.332 	 | Validation Loss: 0.772
Epoch 6 done in 148.10 seconds. 	 | Training Loss: 1.212 	 | Validation Loss: 0.687
Epoch 7 done in 148.21 seconds. 	 | Training Loss: 1.108 	 | Validation Loss: 0.642
Epoch 8 done in 147.74 seconds. 	 | Training Loss: 1.063 	 | Validation Loss: 0.606
Epoch 9 done in 147.65 seconds. 	 | Training Loss: 1.006 	 | Validation Loss: 0.548
Epoch 10 done in 154.56 seconds. 	 | Training Loss: 0.988 	 | Validation Loss: 0.543
Epoch 11 done in 152.65 seconds. 	 | Training Loss: 0.923 	 | Validation Loss: 0.522
Epoch 12 done in 153.34 seconds. 	 | Training Loss: 0.914 	 | Validation L

<All keys matched successfully>

In [13]:
# load best model
model_tl.load_state_dict(torch.load('model_xlearning.pt'))

RuntimeError: ignored

### Model Testing 

In [61]:
def test(loaders, model, criterion, device):
    # monitor test loss and accuracy
    test_loss = 0.
    correct = 0.
    total = 0.
    model.eval()
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(loaders['test']):
            # move to GPU
            data, target = data.to(device), target.to(device)
            bs, ncrops, c, h, w = data.size()
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(data.view(-1, c, h, w)) # fuse batch size and ncrops
            output = output.view(bs, ncrops, -1).mean(1)        
            # calculate the loss
            loss = criterion(output, target)
            # update average test loss 
            test_loss = test_loss + ((1 / (batch_idx + 1)) * (loss.data - test_loss))
            # convert output probabilities to predicted class
            pred = output.data.max(1, keepdim=True)[1]
            # compare predictions to true label
            correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
            total += data.size(0)            
    print('Test Loss: {:.6f}\n'.format(test_loss))
    print('\nTest Accuracy: %2d%% (%2d/%2d)' % (
        100. * correct / total, correct, total))

In [ ]:
test(loaders_dict, model_tl, criterion_tl,device)

### Predictive Model

create pipeline for data prediction
> data -- tranfer -- model prediction -- evaluation score -- output 

In [44]:
class_names = [item[4:].replace("_"," ") for item in data["train"].classes]

def predict_breed(img_path,model,device):
    image = Image.open(img_path).convert("RGB")

    in_tranform = transforms.Compose([
                                      transforms.Resize(img_short_size_resize),
                                      transforms.FiveCrop(img_input_size),
                                      transforms.Lambda(lambda crops: torch.stack([transforms.Compose([
                                                                                                       transforms.ToTensor(),
                                                                                                       transforms.Normalize(mean=norm_mean, std = norm_std)])(crop) for crop in crops]))
                                     ])
    
    scores = model(in_transform(image).to(device)).mean(0)
    output = torch.argmax(score)  
    return output.to("cpu").item(),  F.softmax(score,dim=0).to("cpu").data.numpy()

test_image = dog_files_short[0]
pred,probs = predict_breed(test_image,model_tl, device)
show_image_from_path(test_image,title=class_names[pred])

NameError: ignored

In [ ]:
pip freeze > requirements.txt